In [ ]:
!pip install -Uqq fastbook
!pip install -Uqq duckduckgo_search

In [ ]:

import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *
from duckduckgo_search import ddg_images
from fastcore.all import *
searches = 'forest','desert'

def search_images(term, max_images=20): return L(ddg_images(term, max_results=max_images)).itemgot('image')
urls = search_images(f'{searches[0]} photos', max_images=1)
urls[0]
urls[0]
from fastdownload import download_url
dest = f'{searches[0]}.jpg'
download_url(urls[0], dest, show_progress=False)

In [ ]:
from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)


path = Path(f'{searches[0]}_or_not')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} sun photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} shade photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n=6)


In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)
learn.export('model.pkl')

In [ ]:
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:

img = PILImage.create(btn_upload.data[0])
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred
out_pl
btn_run = widgets.Button(description='Classify')
btn_run

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)



In [ ]:
VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run, out_pl, lbl_pred])


In [1]:
!pip install voila
!jupyter serverextension enable --sys-prefix voila 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 30.9 MB/s eta 0:00:0000:0100:01
Enabling: voila
- Writing config: /opt/conda/etc/jupyter
    - Validating...
      voila 0.4.0 OK
